In [1]:
import numpy as np
from scipy import io

In [3]:
res101 = io.loadmat('../../resnet-feats/CUB/res101.mat')
att_splits = io.loadmat('att_splits.mat')
image_name_list = res101['image_files']
prior_matrix = att_splits['att']
class_labels = res101['labels']

train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

image_names = np.array(['/'.join(y[0][0].split('/')[-2:]) for y in image_name_list])
train_img_names = image_names[np.squeeze(att_splits[train_loc]-1)]
val_img_names = image_names[np.squeeze(att_splits[val_loc]-1)]
test_img_names = image_names[np.squeeze(att_splits[test_loc]-1)]

name_id_path = 'images.txt'
name_id = open(name_id_path).read().splitlines()

id2name = {}
for img in name_id:
    id2name[img.split(' ')[0]] = img.split(' ')[1]

imgattr_labels = open('attributes/image_attribute_labels.txt', 'r')
imgattr_labels = imgattr_labels.readlines()

attr_dict = {}
for i,line in enumerate(imgattr_labels):
    x=line.split(' ')
    if i%312==0:
        sample_labels = []
        sample_labels.append(int(x[2]))
    else:
        sample_labels.append(int(x[2]))
    if i!=0 and (i+1)%312==0:
        attr_dict[id2name[x[0]]] = np.array(sample_labels)

IOError: [Errno 2] No such file or directory: 'attributes/image_attribute_labels.txt'

In [ ]:
train_att_mat=np.zeros([len(train_img_names), 312])

for i in range(train_att_mat.shape[0]):
    train_att_mat[i] = attr_dict[train_img_names[i]]

test_classes = class_labels[np.squeeze(att_splits[test_loc]-1)]
prior_matrix_ts = prior_matrix[:,(np.unique(test_classes)-1)]

In [ ]:
corr_train = np.corrcoef(train_att_mat.transpose())
nans = np.isnan(corr_train)
corr_train[nans] = 0
corr_test = np.corrcoef(prior_matrix_ts)
nans = np.isnan(corr_test)
corr_test[nans] = 0
def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)
dis_corr = diff_corr(corr_train, corr_test)

In [5]:
dis_corr_triu = np.triu(dis_corr,1)
loc = np.unravel_index(np.argsort(-dis_corr_triu, axis=None)[:100], dis_corr_triu.shape)
dis_corr_triu[loc]

array([0.94720618, 0.89724458, 0.89637436, 0.86777269, 0.84916951,
       0.83477524, 0.8310033 , 0.8290578 , 0.82309792, 0.81771457,
       0.81414264, 0.81079432, 0.79156981, 0.78684107, 0.78583007,
       0.78447994, 0.78103757, 0.779318  , 0.77686652, 0.76722963,
       0.76572104, 0.76397167, 0.76078963, 0.75088942, 0.75068951,
       0.7467109 , 0.7426252 , 0.74223388, 0.7359778 , 0.73195772,
       0.72142544, 0.71855233, 0.71669996, 0.71512536, 0.70980274,
       0.7090486 , 0.69878227, 0.69351539, 0.69349986, 0.69168674,
       0.69094441, 0.68991596, 0.68944856, 0.68858182, 0.68843607,
       0.68699016, 0.68353914, 0.6749725 , 0.67334315, 0.67303114,
       0.66795333, 0.66397269, 0.66330133, 0.66052428, 0.65937771,
       0.65507904, 0.64934704, 0.64885098, 0.64554852, 0.64419151,
       0.64322316, 0.64183873, 0.64134972, 0.64108514, 0.64050374,
       0.63185569, 0.62938038, 0.62836847, 0.62677021, 0.62223929,
       0.6218658 , 0.6192955 , 0.61531467, 0.61481853, 0.61366

In [6]:
dis_corr.mean()

0.04704171085922005

<br>
<br>

---
### New split with more correlation shift

In [4]:
res101 = io.loadmat('../../resnet-feats/CUB/res101.mat')
att_splits = io.loadmat('att_splits.mat')
image_name_list = res101['image_files']
prior_matrix = att_splits['att']
class_labels = res101['labels']

train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

image_names = np.array(['/'.join(y[0][0].split('/')[-2:]) for y in image_name_list])
train_img_names = image_names[np.squeeze(att_splits[train_loc]-1)]
val_img_names = image_names[np.squeeze(att_splits[val_loc]-1)]
test_img_names = image_names[np.squeeze(att_splits[test_loc]-1)]

name_id_path = 'images.txt'
name_id = open(name_id_path).read().splitlines()

id2name = {}
for img in name_id:
    id2name[img.split(' ')[0]] = img.split(' ')[1]

In [5]:
imgattr_labels = np.load('attributes/image_attribute_labels.npy', allow_pickle=True).item()
class_names_instance = [x.split('/')[0] for x in image_names]
all_classes = list(np.unique(class_names_instance))

def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)

In [19]:
def get_corr_shift(imgattr_labels, class_names_instance, train_class, test_class):
    test_att_mat = []
    train_att_mat = []
    for i, cls in enumerate(class_names_instance):
        if cls in train_class:
            train_att_mat.append(imgattr_labels[i+1])
        elif cls in test_class:
            test_att_mat.append(imgattr_labels[i+1])
    print '#Train instances = ' + str(len(train_att_mat)) + '\n#Test instances = ' + str(len(test_att_mat))
    
    train_att_mat = np.array(train_att_mat).transpose()
    test_att_mat = np.array(test_att_mat).transpose()
    
    corr_train = np.corrcoef(train_att_mat)
    corr_train[np.isnan(corr_train)] = 0.
    corr_test = np.corrcoef(test_att_mat)
    corr_test[np.isnan(corr_test)] = 0.
    
    dis_corr = diff_corr(corr_train, corr_test)
    
    # correlation shift score: 
    # 1) mean
    # corr_shift_score = np.mean(dis_corr)
    
    # 2) average of top n%
    dis_corr_array = dis_corr.flatten()
    top_percentage = 50
    num_elements = int((top_percentage/100.)*len(dis_corr_array))
    corr_shift_score = np.mean(dis_corr_array[np.argsort(dis_corr_array)[-num_elements:]])
    
    return corr_shift_score

In [36]:
import time

selected_classes = []
remaining_classes = all_classes[:]

start_time = time.time()
for i in range(50):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(imgattr_labels, class_names_instance, new_rem, new_sel)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + selected_classes[-1] + ' -> ' + str(best_score) + ' :' + str(time.time() - start_time)+ 's' 

test_set = selected_classes[:]
train_val = remaining_classes[:]

1) 069.Rufous_Hummingbird -> 0.11385577439266796 :62.9130020142s
2) 057.Rose_breasted_Grosbeak -> 0.11156773006704038 :121.234529018s
3) 191.Red_headed_Woodpecker -> 0.10561692026763937 :179.450778008s
4) 188.Pileated_Woodpecker -> 0.10051679558792562 :237.526015043s
5) 087.Mallard -> 0.09600760215692818 :294.862924099s
6) 192.Downy_Woodpecker -> 0.09312027812516693 :351.022908211s
7) 189.Red_bellied_Woodpecker -> 0.09082375301595 :407.015137196s
8) 083.White_breasted_Kingfisher -> 0.08879891791869768 :462.514536142s
9) 190.Red_cockaded_Woodpecker -> 0.08723255938340584 :517.859834194s
10) 187.American_Three_toed_Woodpecker -> 0.0860217048244411 :572.796858072s
11) 081.Pied_Kingfisher -> 0.08452434396219373 :627.438321114s
12) 159.Black_and_white_Warbler -> 0.08308189489606979 :681.716769218s
13) 044.Frigatebird -> 0.08150842873157368 :735.546095133s
14) 013.Bobolink -> 0.08002348456044096 :789.646750212s
15) 007.Parakeet_Auklet -> 0.07855512642919746 :843.863104105s
16) 006.Least_Aukl

In [44]:
train_val = all_classes[:]
for cls in all_classes:
    if cls in test_set:
        train_val.remove(cls) 
len(train_val)

150

In [50]:
selected_classes = []
remaining_classes = train_val[:]
for i in range(50):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(imgattr_labels, class_names_instance, new_rem, test_set)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + selected_classes[-1] + ' -> ' + str(best_score)
train_set = remaining_classes[:]
val_set = selected_classes[:]

1) 016.Painted_Bunting -> 0.0620539070869597
2) 067.Anna_Hummingbird -> 0.06236520665263801
3) 080.Green_Kingfisher -> 0.06267693606476903
4) 068.Ruby_throated_Hummingbird -> 0.06303446912212289
5) 015.Lazuli_Bunting -> 0.06333623229564776
6) 073.Blue_Jay -> 0.06362335669509035
7) 164.Cerulean_Warbler -> 0.0639269379487011
8) 074.Florida_Jay -> 0.06423630776224475
9) 123.Henslow_Sparrow -> 0.06453022977420048
10) 148.Green_tailed_Towhee -> 0.064846930696718
11) 018.Spotted_Catbird -> 0.06509260691862453
12) 151.Black_capped_Vireo -> 0.0653429401993996
13) 110.Geococcyx -> 0.06560973548435017
14) 055.Evening_Grosbeak -> 0.06587031561541065
15) 158.Bay_breasted_Warbler -> 0.06613818242288234
16) 171.Myrtle_Warbler -> 0.0664175589186118
17) 036.Northern_Flicker -> 0.06668104937227212
18) 099.Ovenbird -> 0.06695580762309218
19) 041.Scissor_tailed_Flycatcher -> 0.06723031110717696
20) 003.Sooty_Albatross -> 0.06747286107511151
21) 155.Warbling_Vireo -> 0.06771935315347377
22) 165.Chestnut_s

In [8]:
train_classes = list(np.unique([cls.split('/')[0] for cls in train_img_names]))
val_classes = list(np.unique([cls.split('/')[0] for cls in val_img_names]))
test_classes = list(np.unique([cls.split('/')[0] for cls in test_img_names]))

In [20]:
get_corr_shift(imgattr_labels, class_names_instance, train_classes, test_classes)

#Train instances = 5875
#Test instances = 2967


0.036478522374301846

In [59]:
split_dict = {}
split_dict['train_cls'] = train_set
split_dict['val_cls'] = val_set
split_dict['test_cls'] = test_set

import pickle
with open('cub_cs_split.npy', 'wb') as fp:
    np.save(fp, split_dict)

In [12]:
sd = np.load('cub_cs_split.npy', allow_pickle=True).item()
sd

{'test_cls': [u'069.Rufous_Hummingbird',
  u'057.Rose_breasted_Grosbeak',
  u'191.Red_headed_Woodpecker',
  u'188.Pileated_Woodpecker',
  u'087.Mallard',
  u'192.Downy_Woodpecker',
  u'189.Red_bellied_Woodpecker',
  u'083.White_breasted_Kingfisher',
  u'190.Red_cockaded_Woodpecker',
  u'187.American_Three_toed_Woodpecker',
  u'081.Pied_Kingfisher',
  u'159.Black_and_white_Warbler',
  u'044.Frigatebird',
  u'013.Bobolink',
  u'007.Parakeet_Auklet',
  u'006.Least_Auklet',
  u'008.Rhinoceros_Auklet',
  u'005.Crested_Auklet',
  u'106.Horned_Puffin',
  u'108.White_necked_Raven',
  u'053.Western_Grebe',
  u'050.Eared_Grebe',
  u'089.Hooded_Merganser',
  u'090.Red_breasted_Merganser',
  u'086.Pacific_Loon',
  u'051.Horned_Grebe',
  u'058.Pigeon_Guillemot',
  u'142.Black_Tern',
  u'048.European_Goldfinch',
  u'019.Gray_Catbird',
  u'093.Clark_Nutcracker',
  u'112.Great_Grey_Shrike',
  u'111.Loggerhead_Shrike',
  u'185.Bohemian_Waxwing',
  u'186.Cedar_Waxwing',
  u'078.Gray_Kingbird',
  u'136.B

In [22]:
get_corr_shift(imgattr_labels, class_names_instance, sd['train_cls'], sd['val_cls'])

#Train instances = 5901
#Test instances = 2958


0.06426110509185505

In [23]:
5901 + 2958

8859